In [3]:
import scipy.sparse

## Remove video that have 0 tokens in the new vocab

In [4]:
S = scipy.sparse.load_npz('/dlabdata1/youtube_large/olam/matrices/S_final2.npz')

In [9]:
row_with_tokens = []
for i in range(S.shape[0]):
    if S[i, :].count_nonzero() != 0:
        row_with_tokens.append(i)

In [10]:
len(row_with_tokens)

68566847

In [11]:
S

<68638982x663127 sparse matrix of type '<class 'numpy.uint8'>'
	with 1251231562 stored elements in Compressed Sparse Row format>

In [12]:
scipy.sparse.save_npz('/dlabdata1/youtube_large/olam/matrices/S_final3.npz', S[row_with_tokens,:])